In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import pylab as plt
from collections import Counter
import datetime

%matplotlib notebook

# flag to control where data is loaded to
mode = 'sqlite3'

# sqlite stuff
if mode == 'sqlite3':
    import sqlite3
    conn = sqlite3.connect("../../../data/sample_data.db")
    cursor = conn.cursor()
elif mode == 'psycopg2': # alternatively use postgresql
    import psycopg2
    connect_str = "dbname='sepsis' user='sepsis' host='localhost' " + \
                  "password='sepsis'"
    conn = psycopg2.connect(connect_str)
    cursor = conn.cursor()

qlog_conn = sqlite3.connect('../../../data/kfp_log.db')
q_cursor = qlog_conn.cursor()

start = datetime.datetime.now()
# because names are created as case sensistive in postgres, must be quoted...
# should probably fix that...
sql = '''
SELECT d."SubjectId",
    d."EncounterId",
    d."Source",
    -- d."StartDate",
    d."Code",
    d."Type",
     MAX("FlowsheetValue") AS MaxScore,
     AVG("FlowsheetValue") AS MeanScore,
     MIN("FlowsheetValue") AS MinScore,
     COUNT("FlowsheetValue") AS NumLoggedScores
 FROM diagnoses d
 LEFT JOIN flowsheet f
 ON d."EncounterId" = f."EncounterId"
-- GROUP BY d."SubjectId", d."EncounterId", d."Source", d."StartDate", d."Code", d."Type"
 GROUP BY d."SubjectId", d."EncounterId", d."Source", d."Code", d."Type"
 ORDER BY NumLoggedScores DESC
 limit 100
'''

#sql = 'SELECT subjectid, encounterid, source, code, type FROM "diagnoses" limit 100'


#df = pd.read_sql(sql,conn, index_col=['SubjectId', 'EncounterId'])
df = pd.read_sql(sql,conn)
#df['StartDate'] = df['StartDate'].astype('datetime64')
# print(df.dtypes)
# print('Elapsed time:', datetime.datetime.now() - start)
df.head()

# sqlite - 42 to 60 seconds
# postgres - 29 seconds

In [ ]:
def show_timings(df):
    q = pd.read_sql("SELECT * FROM kfp_log order by fauxify_end",qlog_conn)
    print('Method used     :', q.tail(1)['faux_method'].iloc[0])
    print('Time for query  :', (pd.to_datetime(q.tail(1)['query_end']) - pd.to_datetime(q.tail(1)['query_start'])).iloc[0])
    print('Time for fauxify:', (pd.to_datetime(q.tail(1)['fauxify_end']) - pd.to_datetime(q.tail(1)['fauxify_start'])).iloc[0])
    print('Size of dataset :', len(df), 'rows')

In [ ]:
from importlib import reload
from kungfauxpandas import KungFauxPandas, TrivialPlugin, DataSynthesizerPlugin, KDEPlugin
kfpd = KungFauxPandas()

In [ ]:
kfpd.plugin = DataSynthesizerPlugin(mode='correlated_attribute_mode', verbose=False)
#fdf=kfpd.read_sql(sql,conn, index_col=['SubjectId', 'EncounterId'])
fdf=kfpd.read_sql(sql,conn)#, fauxify = {'categorical_attributes' : {'Source' : True}})
#fdf = kfpd.plugin.fauxify(df)
fdf.head()

#fdf=kfpd.read_sql(sql,conn, index_col=['SubjectId', 'EncounterId'])
#fdf=kfpd.read_sql(sql,conn, fauxify = {'categorical_attributes' : {'Source' : True}})
#df=pd.read_sql(sql + ' 10',conn)
#fdf = kfpd.plugin.fauxify(df)#, categorical_attributes = {'Source' : True, 'Code': True, 'Type': True})

In [ ]:
kfpd.plugin = TrivialPlugin()
for n in ['10', '100', '1000', '10000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

fdf.head()

In [ ]:
kfpd.verbose

In [ ]:
kfpd.plugin = KDEPlugin()
for n in ['10', '100', '1000', '10000']:
    fdf=kfpd.read_sql(sql + n,conn)
    show_timings(fdf)

In [ ]:
fdf

In [ ]:
col = 'Code'
out_dict = dict()

colfact = df[col].factorize()
cc=Counter(colfact[0])
 
# convert from counts to proportions

for key in cc:
     cc[key] = cc[key] / len(df)

fakes = choice(elements,p=weights, replace=True, size=len(df))

out_dict[col] = [colfact[1][xx] for xx in fakes]


In [ ]:
len(cc.values()), len(df), len(cc)/len(df)

In [ ]:
col = 'Code'
out_dict = dict()

colfact = df[col].factorize()
cc=Counter(colfact[0])
 
# convert from counts to proportions

for key in cc:
     cc[key] = cc[key] / len(df)

fakes = choice(elements,p=weights, replace=True, size=len(df))

out_dict[col] = [colfact[1][xx] for xx in fakes]
#out_dict

In [ ]:
col = 'SubjectId'
kd = stats.gaussian_kde(df[col], bw_method='silverman')
out_dict[col]=np.int64(kd.resample()[0])


In [ ]:
df.head()

In [ ]:
pd.crosstab(df.Codeode, df.squishcode)

In [ ]:
np.corrcoef(df.Code, df.squishcode)

In [ ]:
sdf = df.sample(50000)
for thiscol in sdf.columns:
    if sdf[thiscol].dtype=='object':
        print('Converting column ', thiscol)
        sdf[thiscol] = sdf[thiscol].factorize()[0]
    
#np.cov(sdf)

In [ ]:
cc = np.corrcoef(sdf.transpose())
#cc = np.cov(sdf.transpose())
#cc[5,1]
plt.imshow(cc,cmap='inferno')
plt.colorbar()

In [ ]:
#sdf.head()
#help(np.correlate)
df.iloc[3]

In [ ]:
from statsmodels.nonparametric import kernel_density as kd

In [ ]:
woo = kd.KDEMultivariate(np.array(sdf.iloc[:,[2,4,9]]), var_type=3*'u')
#help(kd.KDEMultivariate)

In [ ]:
np.array(data=sdf.sample(2000).iloc[:,[2,4,9]])

In [ ]:
xx = range(40)
bb = list(itertools.product(xx,xx,xx))

In [ ]:
np.array(sdf.iloc[2]).shape

In [ ]:
from scipy.optimize import fsolve
import statsmodels.api as sm
import numpy as np

# fit
kde = woo#  # ... you already did this

# sample
u = np.random.random()

# 1-d root-finding
def func(x):
    return kde.cdf([x]) - u
#sample_x = brentq(func, -99999999, 99999999)  # read brentq-docs about these constants
                                              # constants need to be sign-changing for the function


In [ ]:
#u = np.random.random()
#u
#sample_x = brentq(func, -99999999, 99999999)

In [ ]:
def func(x):
    return kde.cdf([x]) - u

x0=[92,4,5,3,6,7,8,9,10,11]


In [ ]:
from scipy.optimize import minimize
darf = minimize(func,np.array(x0))
print(darf)

In [ ]:
x0, func(x0)


In [ ]:
func([0,0,0,0,0,3,0,0,0,0])

In [ ]:
bork = np.mgrid[0:10,0:10, 0:10]

In [ ]:
xx = range(4)

import itertools
ins = list(itertools.product(xx,xx,xx,xx,xx,xx,xx,xx,xx,xx))

vals = [func(i) for i in ins[1004:2004]]
func(ins[1004:2004])

In [ ]:
func(bork[32532])

In [ ]:
u


In [ ]:
#kde.cdf(bork[9000:10000])
func(x0)

In [ ]:
list(bork[0])

In [ ]:
x0

In [ ]:
import statsmodels.api as sm
nobs = 300
np.random.seed(1234)  # Seed random generator
c1 = np.random.normal(size=(nobs,1))
c2 = np.random.normal(2, 1, size=(nobs,1))
  
#Estimate a bivariate distribution and display the bandwidth found:
   
#dens_u = sm.nonparametric.KDEMultivariate(data=[c1,c2], var_type='cc', bw='normal_reference')
#dens_u.bw

woo = sm.nonparametric.KDEMultivariate(data=sdf.iloc[:,[2,4,9]], var_type=3*'u')



In [ ]:
woo.cdf()

In [ ]:
len(sdf)

In [ ]:
len(set(sdf.iloc[:,9]))

In [ ]:
np.corrcoef(sdf.iloc[:,[2,9]])